In [3]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
table_name = "d_promo"
table_temp = "d_promo_temp"

In [3]:
sql = \
"""
DECLARE @from DATE = '2021-10-01'
DECLARE @to DATE = '2021-12-31'
SELECT DISTINCT
        ord.OrderDate,  
       ord.BranchID,
	   ord.OrigOrderNbr,
       ord.OrderNbr,
       d.InvtID,
       d.LineRef,
	   d.FreeItem,
	   --dis.FreeItemQty,
       sq.TypeDiscount,
       DiscAmt = CASE
                     WHEN dis.DiscType = 'L' THEN
                         d.DiscAmt
                     WHEN dis.DiscType = 'G' THEN
                         d.GroupDiscAmt1
                     WHEN dis.DiscType = 'D' THEN
                         d.DocDiscAmt
                 END,
       DiscPct = CASE
                     WHEN dis.DiscType = 'L' THEN
                         d.DiscPct
                     WHEN dis.DiscType = 'G' THEN
                         d.GroupDiscPct1
                     WHEN dis.DiscType = 'D' THEN
                         d.DocDiscAmt
                 END,
       sq.DiscIDPN,
       --sq.DiscID,
       --sq.DiscSeq,
       --dis.SOLineRef,
       sq.Descr,
	   --dis.GroupRefLineRef,
	   sq.AccumulateID
FROM dbo.OM_SalesOrd ord WITH (NOLOCK)
    INNER JOIN dbo.OM_SalesOrdDet d WITH (NOLOCK)
        ON d.BranchID = ord.BranchID
           AND d.OrderNbr = ord.OrderNbr
    INNER JOIN dbo.OM_OrdDisc dis WITH (NOLOCK)
        ON dis.BranchID = d.BranchID
           AND dis.OrderNbr = d.OrderNbr
           AND ( d.LineRef = dis.SOLineRef or d.LineRef IN(SELECT part FROM dbo.fr_SplitStringMAX(dis.GroupRefLineRef, ',')) )
		   --AND ( d.LineRef IN(SELECT part FROM dbo.fr_SplitStringMAX(dis.GroupRefLineRef, ',')) )
    INNER JOIN dbo.OM_DiscSeq sq WITH (NOLOCK)
        ON sq.DiscID = dis.DiscID
           AND sq.DiscSeq = dis.DiscSeq
WHERE CAST(ord.OrderDate AS DATE) >= @from
and CAST(ord.OrderDate AS DATE) <= @to
"""

In [1]:
sql = \
"""
DECLARE @from DATE = '2022-05-01'
DECLARE @to DATE = '2022-08-31'
SELECT DISTINCT
        ord.OrderDate,  
       ord.BranchID,
	   ord.OrigOrderNbr,
       ord.OrderNbr,
       d.InvtID,
       d.LineRef,
	   d.FreeItem,
	   --dis.FreeItemQty,
       sq.TypeDiscount,
       DiscAmt = CASE
                     WHEN dis.DiscType = 'L' THEN
                         d.DiscAmt
                     WHEN dis.DiscType = 'G' THEN
                         d.GroupDiscAmt1
                     WHEN dis.DiscType = 'D' THEN
                         d.DocDiscAmt
                 END,
       DiscPct = CASE
                     WHEN dis.DiscType = 'L' THEN
                         d.DiscPct
                     WHEN dis.DiscType = 'G' THEN
                         d.GroupDiscPct1
                     WHEN dis.DiscType = 'D' THEN
                         d.DocDiscAmt
                 END,
       sq.DiscIDPN,
	   case when dis.SOLineRef != '' then 'item' else '' END as tangitem,
		case when dis.DiscAmt != 0 then 'tangtien' else '' END as tangtien,
       --sq.DiscID,
       --sq.DiscSeq,
       --dis.SOLineRef,
       sq.Descr,
	   --dis.GroupRefLineRef,
	   sq.AccumulateID
FROM dbo.OM_SalesOrd ord WITH (NOLOCK)
    INNER JOIN dbo.OM_SalesOrdDet d WITH (NOLOCK)
        ON d.BranchID = ord.BranchID
           AND d.OrderNbr = ord.OrderNbr
    INNER JOIN dbo.OM_OrdDisc dis WITH (NOLOCK)
        ON dis.BranchID = d.BranchID
           AND dis.OrderNbr = d.OrderNbr
           AND ( d.LineRef = dis.SOLineRef or d.LineRef IN(SELECT part FROM dbo.fr_SplitStringMAX(dis.GroupRefLineRef, ',')) )
		   --AND ( d.LineRef IN(SELECT part FROM dbo.fr_SplitStringMAX(dis.GroupRefLineRef, ',')) )
    INNER JOIN dbo.OM_DiscSeq sq WITH (NOLOCK)
        ON sq.DiscID = dis.DiscID
           AND sq.DiscSeq = dis.DiscSeq
WHERE CAST(ord.OrderDate AS DATE) >= @from
and CAST(ord.OrderDate AS DATE) <= @to
"""

In [4]:
df0 = get_ms_df(sql)

In [24]:
df0.shape

(158614, 15)

In [5]:
df = df0.copy()

In [6]:
df.columns = lower_col(df)

In [7]:
# checkdup(df,2, subset=[''])

In [8]:
dk = checkdup(df, 2, ['branchid', 'ordernbr', 'lineref'])

In [9]:
# df[dk].to_clipboard()

In [10]:
df['duplineref'] = np.where(dk, "y","n")

In [11]:
vc(df, 'duplineref')

,duplineref,counts
0,n,155147
1,y,3467


In [12]:
dk = df['duplineref'] == 'n'

In [13]:
df1 = df[dk].copy()

In [15]:
df1.columns

Index(['orderdate', 'branchid', 'origordernbr', 'ordernbr', 'invtid',
       'lineref', 'freeitem', 'typediscount', 'discamt', 'discpct', 'discidpn',
       'tangitem', 'tangtien', 'descr', 'accumulateid', 'duplineref'],
      dtype='object')

In [17]:
drop_cols(df1, ['tangitem', 'tangtien'])

In [16]:
df1['inserted_at'] = datetime.now()

In [18]:
bq_values_insert(df1, "d_promo", 2)

1it [00:15, 15.41s/it]


In [20]:
df1.shape

(155147, 15)

In [21]:
dk2 = checkdup(df, 2, ['branchid', 'ordernbr', 'lineref','discidpn'])

In [22]:
df['dup_discidpn'] = np.where(dk2, "y","n")

In [23]:
vc(df, 'dup_discidpn')

,dup_discidpn,counts
0,n,158432
1,y,182


In [25]:
dk3 = df['dup_discidpn'] == 'y'

In [26]:
df_dup_discidpn = df[dk3].copy()

In [27]:
# df_dup_discidpn.to_clipboard()

In [28]:
dk1 = df_dup_discidpn['discamt'] > 0
dk2 = df_dup_discidpn['tangtien'] == ''
dk3 = df_dup_discidpn['typediscount'] == 'AC'

In [29]:
#Loai bo cac case trung discidpn
df_dup_discidpn = df_dup_discidpn[~(dk1&dk2&dk3)]

In [30]:
df_dup_discidpn['duplineref'] = 'n'

In [31]:
drop_cols(df_dup_discidpn, ['tangitem', 'tangtien', 'dup_discidpn'])

In [32]:
df_dup_discidpn.shape

(91, 14)

In [60]:
# df_dup_discidpn.head()

In [33]:
#loc bo 43 don trong 86 don
bq_values_insert(df_dup_discidpn, "d_promo", 2)

1it [00:05,  5.48s/it]


In [34]:
dk1 = df['duplineref'] == 'y'
dk2 = df['dup_discidpn'] == 'n'

In [35]:
df_dup_discidpn = df[dk1&dk2].copy()

In [36]:
# df_dup_discidpn['join'] = df_dup_discidpn['branchid'] + df_dup_discidpn['ordernbr'] + df_dup_discidpn['lineref']

In [37]:
# df_dup_discidpn.to_clipboard()

In [38]:
dk1 = df_dup_discidpn['discamt'] > 0
dk2 = df_dup_discidpn['tangtien'] == 'tangtien'

In [39]:
df_dup_discidpn_tangtien = df_dup_discidpn[dk1&dk2].copy()

In [40]:
df_dup_discidpn_tangtien.columns

Index(['orderdate', 'branchid', 'origordernbr', 'ordernbr', 'invtid',
       'lineref', 'freeitem', 'typediscount', 'discamt', 'discpct', 'discidpn',
       'tangitem', 'tangtien', 'descr', 'accumulateid', 'duplineref',
       'dup_discidpn'],
      dtype='object')

In [41]:
# df_dup_discidpn_tangtien['join'] = df_dup_discidpn_tangtien['branchid'] + df_dup_discidpn_tangtien['ordernbr'] + df_dup_discidpn_tangtien['lineref']

In [42]:
df_dup_discidpn_tangtien['duplineref'] = 'n'

In [43]:
drop_cols(df_dup_discidpn_tangtien, ['tangitem', 'tangtien', 'dup_discidpn'])
bq_values_insert(df_dup_discidpn_tangtien, "d_promo", 2)

1it [00:05,  5.96s/it]


In [44]:
dk1 = df_dup_discidpn['discamt'] > 0
dk2 = df_dup_discidpn['tangtien'] != 'tangtien'

In [45]:
df_dup_discidpn_tangitem = df_dup_discidpn[dk1&dk2].copy()

In [46]:
# checkdup(df_dup_discidpn_tangitem, 2, ['branchid', 'ordernbr', 'lineref']).sum()

In [47]:
# df_dup_discidpn_tangitem.to_clipboard()

In [48]:
drop_cols(df_dup_discidpn_tangitem, ['tangitem', 'tangtien', 'dup_discidpn'])
bq_values_insert(df_dup_discidpn_tangitem, "d_promo", 2)

1it [00:03,  3.99s/it]


In [49]:
# df_dup_discidpn_tangitem['join'] = df_dup_discidpn_tangitem['branchid'] + df_dup_discidpn_tangitem['ordernbr'] + df_dup_discidpn_tangitem['lineref']

In [50]:
# df_dup_discidpn_tangitem.to_clipboard()

In [51]:
dk1 = df_dup_discidpn['discamt'] == 0

In [52]:
df_dup_discidpn_tangitem_only = df_dup_discidpn[dk1].copy()

In [54]:
# df_dup_discidpn_tangitem_only

In [55]:
# df_dup_discidpn_tangitem_only['join'] = df_dup_discidpn_tangitem_only['branchid'] + df_dup_discidpn_tangitem_only['ordernbr'] + df_dup_discidpn_tangitem_only['lineref']

In [56]:
# df_dup_discidpn_tangitem_only.to_clipboard()

In [57]:
# case 1 line ref tham gia 2 ctr tặng sản phẩm, xuất hiện lần 1 duplineref = n
df_dup_discidpn_tangitem_only_1 = df_dup_discidpn_tangitem_only[checkdup(df_dup_discidpn_tangitem_only, 1, ['branchid', 'ordernbr', 'lineref'])].copy()
df_dup_discidpn_tangitem_only_2 = df_dup_discidpn_tangitem_only[checkdup(df_dup_discidpn_tangitem_only, 3, ['branchid', 'ordernbr', 'lineref'])].copy()
# nam trong df_dup_discidpn_tangtien => duplineref = n
df_dup_discidpn_tangitem_only_3 = df_dup_discidpn_tangitem_only[~checkdup(df_dup_discidpn_tangitem_only, 2, ['branchid', 'ordernbr', 'lineref'])].copy()

In [58]:
print(df_dup_discidpn_tangitem_only_1.shape)
print(df_dup_discidpn_tangitem_only_2.shape)
print(df_dup_discidpn_tangitem_only_3.shape)

(1502, 17)
(1502, 17)
(17, 17)


In [59]:
df_dup_discidpn_tangitem_only_1.columns

Index(['orderdate', 'branchid', 'origordernbr', 'ordernbr', 'invtid',
       'lineref', 'freeitem', 'typediscount', 'discamt', 'discpct', 'discidpn',
       'tangitem', 'tangtien', 'descr', 'accumulateid', 'duplineref',
       'dup_discidpn'],
      dtype='object')

In [61]:
df_dup_discidpn_tangitem_only_1['duplineref'] = 'n'
drop_cols(df_dup_discidpn_tangitem_only_1, ['tangitem', 'tangtien', 'dup_discidpn'])
bq_values_insert(df_dup_discidpn_tangitem_only_1, "d_promo", 2)

1it [00:04,  4.73s/it]


In [62]:
df_dup_discidpn_tangitem_only_2['duplineref'] = 'y'
drop_cols(df_dup_discidpn_tangitem_only_2, ['tangitem', 'tangtien', 'dup_discidpn'])
bq_values_insert(df_dup_discidpn_tangitem_only_2, "d_promo", 2)

1it [00:09,  9.80s/it]


In [63]:
df_dup_discidpn_tangitem_only_3['duplineref'] = 'y'
drop_cols(df_dup_discidpn_tangitem_only_3, ['tangitem', 'tangtien', 'dup_discidpn'])
bq_values_insert(df_dup_discidpn_tangitem_only_3, "d_promo", 2)

1it [00:06,  6.05s/it]


In [12]:
# df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(orderdate)
CLUSTER BY branchid,ordernbr,lineref
"""
execute_bq_query(sql)

1it [00:17, 17.84s/it]


In [7]:
# print(sql)

In [13]:
# df = get_ms_df(sql)
# df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_name}", 2)

1it [00:19, 19.78s/it]


In [14]:
df.shape

(200206, 15)

In [10]:
bq_values_insert(df, f"{table_name}", 2)

1it [00:28, 28.29s/it]


In [13]:
sql = \
"""
select mahd, macongtycn, lineref from biteam.f_sales where date(ngaychungtu) < '2022-01-01'
"""

In [14]:
df = get_bq_df(sql)

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(462038, 3)

In [4]:
sql2 = """
select ordernbr as mahd, branchid as macongtycn, lineref from `spatial-vision-343005.biteam.d_promo` where date(orderdate) >= '2022-01-01'
"""

In [5]:
df2= get_bq_df(sql2)

In [9]:
df2[checkdup(df2,2)]

,mahd,macongtycn,lineref
71,HD1-0422-00863,MR0010,00009
76,HD1-0422-00863,MR0010,00009
151,HD0-0122-12264,MR0001,00018
153,HD0-0122-12264,MR0001,00018
534,HD0-0322-25928,MR0001,00009
...,...,...,...
198962,HD6-0122-05374,MR0016,00016
198979,HD5-0322-00045,MR0015,00016
198980,HD5-0322-00045,MR0015,00016
198994,HD1-0222-00223,MR0010,00016


In [21]:
df2.drop_duplicates(inplace=True)

In [23]:
dfs_diff(df,df2).to_clipboard()

In [1]:
with open("date.csv") as f:
    lines = f.readlines()

In [3]:
for l in lines:
    print(l)

from,to

2022-01-01,2022-01-31


In [9]:
# bq_values_insert(df, f"{table_name}", 2)

1it [02:04, 124.56s/it]


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='DV'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [2]:
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")

In [3]:
from_tb = "OM_Delivery"

In [4]:
sql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
BatNbr,
OrderNbr,
Sequence,
SlsperID,
Status,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime
from {from_tb}
where Crtd_Datetime >= @from
"""

In [ ]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
BatNbr,
OrderNbr,
Sequence,
SlsperID,
Status,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime
from {from_tb}
where LUpd_DateTime >= @from
and Crtd_DateTime < @from
"""

In [2]:
table_name = "sync_dms_dv"
table_temp = "sync_dms_dv_temp"

In [1]:
# print(sql)
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
# df['inserted_At']
# bq_values_insert(df, f"{table_temp}", 3)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_name};
# CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
# PARTITION BY DATE(crtd_datetime)
# CLUSTER BY branchid,batnbr,ordernbr
# """
# print(sql)
# execute_bq_query(sql)

dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)

bq_values_insert(df, f"{table_name}", 2)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_temp};
# """
# execute_bq_query(sql)


In [ ]:
#UPDATE
df_update = get_ms_df(usql)
df_update.columns = lower_col(df_update)
# df_update.columns
df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
df_update1 = df_update['crtd_datetime']
df_update1.drop_duplicates(inplace=True)
tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
# tpl_dt
df_update1 = df_update['branchid'] + df_update['batnbr'] + df_update['ordernbr'] + df_update['sequence']
df_update1.drop_duplicates(inplace=True)
tpl_pk = tuple(df_update1.to_list()) + ('',)
del_sql = \
f"""
DELETE FROM biteam.{table_name}
WHERE
DATE(crtd_datetime) in {tpl_dt}
AND concat(branchid,batnbr,ordernbr,sequence) in {tpl_pk}
"""
print("del_sql ",del_sql)
execute_bq_query(del_sql)
df_update['inserted_at'] = datetime.now()
bq_values_insert(df_update, f"{table_name}", 2)

In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update